In [1]:
!pip install langgraph langchain pandas faiss-cpu langchain-groq sentence-transformers langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.8/145.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.4/412.4 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.31
    Uninstalling langchain-core-0.3.31:
      Successfully uninstalled langchain-core-0.3.31
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.15
    Uninstalling langchain-0.3.15:
      Successfully uninstalled langchain-0.3.15


In [2]:
from google.colab import userdata
groq_api = userdata.get("groq_api_key")

In [3]:
import pandas as pd
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langgraph.graph import END, StateGraph
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from typing import TypedDict, List

In [32]:
df = pd.read_csv("/content/failure reason.csv")
# df.head()

docs = []
for _, row in df.iterrows():
    if pd.notna(row["Failure Reason"]) and pd.notna(row["Solution"]):
        docs.append(Document(
            page_content=row["Failure Reason"],
            metadata={"solution": row["Solution"]}
        ))

In [18]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(docs, embeddings)
retriever = vector_store.as_retriever(search_kwargs={"k": 1})

In [19]:
class AgentState(TypedDict):
    input: str
    context: List[dict]
    response: str

In [20]:
llm = ChatGroq(
    groq_api_key = groq_api,
    temperature=0.3,  # Slightly higher temp for creative generation
    model_name="llama-3.2-1b-preview",
)

In [33]:
def retrieve(state: AgentState):
    try:
        relevant_docs = retriever.invoke(state["input"])
        return {"context": relevant_docs}
    except:
        return {"context": []}

def generate_response(state: AgentState):
    try:
        response_template = """**Error:**\n{Error}\n\n**Solution**\n{Solution}\n\n**Resoulution Steps**\n{Resolution}"""
        # Check if solution exists in CSV
        if state["context"] and "solution" in state["context"][0].metadata:
            context = state["context"][0]
            prompt_template = """
            [INST] Given this error and known solution:
            Error: {error}
            Solution: {solution}

            Generate only Resolution Steps (numbered list of 5 steps) [/INST]
            """
            result = llm.invoke(ChatPromptTemplate.from_template(prompt_template).format(
                error=state["input"],
                solution=context.metadata["solution"]
            )).content
            #parse only resolution steps
            resolution = "\n".join([line for line in result.split("\n") if line.strip() and line[0].isdigit() or line.startswith("-")])
            return {"response": response_template.format(
                Error=state["input"],
                Solution=context.metadata["solution"],
                Resolution=resolution
            )}
        else:
            # Generate everything from scratch
            prompt_template = """
            [INST] As a system administrator, handle this new error:
            Error: {error}

            Generate:
            1. Solution
            2. Resolution Steps [/INST]
            """
            full_response = llm.invoke(ChatPromptTemplate.from_template(prompt_template).format(
                error=state["input"]
            )).content
            #Parse solution and resolution
            sections = full_response.split("\n\n")
            return {"response": response_template.format(
                Error=state["input"],
                Solution=sections[0].replace("1. Solution: ", "").strip(),
                Resolution=sections[1].replace("2. Resolution Steps\n", "").strip()
            )}

    except Exception as e:
        return {"response": f"Error processing request: {str(e)}"}


In [35]:
workflow = StateGraph(AgentState)
workflow.add_node("retrieve", retrieve)
workflow.add_node("generate_response", generate_response)
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "generate_response")
workflow.add_edge("generate_response", END)
agent = workflow.compile()

In [36]:
def get_solution(error_message):
    result = agent.invoke({"input": error_message.strip()})
    return result["response"]

In [37]:
# Test with both cases
print("\n=== Known Error ===")
print(get_solution("API rate limiting not enforced"))
print("\n======\n")
print("=== UnKnown Error ===")
print(get_solution("Disk Read Error"))



=== Known Error ===
**Error:**
API rate limiting not enforced

**Solution**
Implement rate limiting and throttling mechanisms

**Resoulution Steps**
1. **Understand the API's Rate Limiting Policy**: Familiarize yourself with the API's rate limiting policy, including the allowed rate of requests per user, per IP address, or per time period. This will help you determine the maximum number of requests that can be made within a given timeframe.
2. **Implement IP Address-Based Rate Limiting**: Use a library or service that allows you to track IP addresses and limit the number of requests from each IP address. This can be done by storing IP addresses in a database and checking for IP address uniqueness before allowing a request.
3. **Use a Throttling Mechanism**: Implement a throttling mechanism that limits the number of requests from a single user or IP address within a given timeframe. This can be done by using a queue-based approach, where requests are stored in a queue and executed in a